# Retail Credit Risk Project

In [155]:
# import packages
from collections import Counter
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
import math
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Data Cleaning

In [156]:
# import dataset
df = pd.read_csv('../data/mmf_data2.csv')
print(len(df.columns))
df.head()

562


,tu_seq_id,TIME_KEY,CUSTTYPE,HIGHRISK,WIDELYHD,STARTUP,TERMCUST,PDLCUST,OLCUST,VISACUST,PROFTYPE,BSL_SB_CUST,BSL_CUST_AIRB,HasAIRBProduct,Widely_Held,NFP,Out_of_Country,SPP_Group_1,SPP_Group_2,Automotive_New_Cars_Dealer,BOBB,OPFBCUST,DOCTOR_DENTIST_IND,RESTAURANT_IND,TRUCKING_IND,RESTAURANT_OR_TRUCKING_IND,SPP_LISTED_IND,SPP_TARGETED_IND,SPP_LISTED_OR_TARGETED_IND,TBSAT01,TBSAT103S,TBSAT33A,TBSAT34B,TBSBC104S,TBSBC33S,TBSBC35S,TBSBC97A,TBSBR34S,TBSG001B,TBSG059S,TBSG202A,TBSG302S,TBSRE24S,TBSRE29S,TBSRE30S,TBSRE36S,TBSRN33S,TBSRN34S,GO170,NA11,CVPRAEP112,CVPRAGG102,CVPRAGG501,CVPRAGG512,CVPRAGG519,CVPRAGG905,CVPRAGG907,CVPRAGG910,CVPRRVLR01,CVPRRVLR07,CVPRTPR103,CVPRTPR212,CVPRTPR301,CVPRTPR312,CVPRTRV04,CVPRTRV12,CVPRTRV14,CVPRWALSHR01,CVPRWALSHR02,cust_any_nsf,cust_card_max_util_6mos,cust_max_dlq_3mos,cust_rev_max_dlq_6mos,cust_sum_dlq_24mos,cust_sum_dlqdays_12mos,d2dbalp_avg_l6m,agri_cust,TBSSC100,CVPRRVLR29,GO138,Go129_yymm,dda_max_open_date,dda_max_Last_Dep_Date,dda_sum_ABM_Dep_Txn_Cnt,dda_sum_Acc_Cr_Bal,dda_sum_Acc_Daily_OD_Int,dda_sum_Acc_Db_Bal,dda_max_Avg_Cr_Bal,dda_max_Avg_Cr_Bal_Prev,dda_min_Avg_Db_Bal,dda_sum_Chargebacks,dda_sum_Dep_Txn_Cnt,dda_sum_Limit,dda_max_Max_Mthly_Bal,dda_min_Min_Mthly_Bal,dda_sum_Night_Dep_Txn_Cnt,dda_sum_OD_Charges,dda_sum_OL_Days,dda_sum_OS_Bal,dda_sum_Qtr_NSF,dda_sum_Qtr_NSF_Prev,dda_sum_Qtr_NSF_Prev_2,dda_sum_Qtr_NSF_Prev_3,dda_sum_Ttl_Dep_Prev,dda_sum_Ttl_Dep_Prev_2,dda_sum_Ttl_Dep_Prev_3,dda_avg_av_db_bal,dda_time_since_open_mos,dda_max_mthly_av_bal,dda_min_mthly_av_bal,dda_av_bal,dda_time_since_dep_days,dda_avg_dly_dep_amt_L90,dda_ratio_cr_bal,lease_tot_osbal,max_ls_age,max_ls_any_bad_12mos,max_ls_any_bad_24mos,max_ls_any_bad_3mos,max_ls_any_bad_6mos,max_ls_any_def_12mos,max_ls_any_def_24mos,max_ls_any_def_3mos,max_ls_any_def_6mos,max_ls_max_dlq_12mos,max_ls_max_dlq_24mos,max_ls_max_dlq_3mos,max_ls_max_dlq_6mos,max_ls_max_dlqdays_12mos,max_ls_max_dlqdays_24mos,max_ls_max_dlqdays_3mos,max_ls_max_dlqdays_6mos,max_ls_months_default,max_ls_num_bad_12mos,max_ls_num_bad_24mos,max_ls_num_bad_3mos,max_ls_num_bad_6mos,max_ls_num_def_12mos,max_ls_num_def_24mos,max_ls_num_def_3mos,max_ls_num_def_6mos,max_ls_num_dlq_12mos,max_ls_num_dlq_24mos,max_ls_num_dlq_3mos,max_ls_num_dlq_6mos,max_ls_num_dlqdays_12mos,max_ls_num_dlqdays_24mos,max_ls_num_dlqdays_3mos,max_ls_num_dlqdays_6mos,mo_tot_osbal,mo_tot_auth_amt,max_mo_DELQ_DAYS,max_mo_age,max_mo_any_bad_12mos,max_mo_any_bad_24mos,max_mo_any_bad_3mos,max_mo_any_bad_6mos,max_mo_any_def_12mos,max_mo_any_def_24mos,max_mo_any_def_3mos,max_mo_any_def_6mos,max_mo_max_dlq_12mos,max_mo_max_dlq_24mos,max_mo_max_dlq_3mos,max_mo_max_dlq_6mos,max_mo_max_dlqdays_12mos,max_mo_max_dlqdays_24mos,max_mo_max_dlqdays_3mos,max_mo_max_dlqdays_6mos,max_mo_num_bad_12mos,max_mo_num_bad_24mos,max_mo_num_bad_3mos,max_mo_num_bad_6mos,max_mo_num_def_12mos,max_mo_num_def_24mos,max_mo_num_def_3mos,max_mo_num_def_6mos,max_mo_num_dlq_12mos,max_mo_num_dlq_24mos,max_mo_num_dlq_3mos,max_mo_num_dlq_6mos,max_mo_num_dlqdays_12mos,max_mo_num_dlqdays_24mos,max_mo_num_dlqdays_3mos,max_mo_num_dlqdays_6mos,max_mo_remainamort,mo_pct_tot_utilz,min_oll_avg_ytd_bal,max_oll__dlq_times,max_oll__days_late,tot_oll_limit,tot_oll_os_bal,max_oll_any_def_24mos,max_oll_any_def_12mos,max_oll_any_def_6mos,max_oll_any_def_3mos,max_oll_any_bad_24mos,max_oll_any_bad_12mos,max_oll_any_bad_6mos,max_oll_any_bad_3mos,max_oll_num_def_24mos,max_oll_num_def_12mos,max_oll_num_def_6mos,max_oll_num_def_3mos,max_oll_num_bad_24mos,max_oll_num_bad_12mos,max_oll_num_bad_6mos,max_oll_num_bad_3mos,max_oll_num_dlq_24mos,max_oll_num_dlq_12mos,max_oll_num_dlq_6mos,max_oll_num_dlq_3mos,max_oll_num_overlimit_24mos,max_oll_num_overlimit_12mos,max_oll_num_overlimit_6mos,max_oll_num_overlimit_3mos,max_oll_term_max_dlq_24mos,max_oll_term_max_dlq_12mos,max_oll_term_max_dlq_6mos,max_oll_term_max_dlq_3mos,max_oll_line_max_dlq_24mos,max_oll_line_max_dlq_12mos,max_oll_line_max_dlq_6mos,max_oll_line_max_dlq_3mos,max_oll_age,max_oll_max_dlqt

In [157]:
# check datatype
print(df.dtypes)

tu_seq_id                         object
TIME_KEY                           int64
CUSTTYPE                          object
HIGHRISK                          object
WIDELYHD                          object
STARTUP                           object
TERMCUST                          object
PDLCUST                           object
OLCUST                            object
VISACUST                          object
PROFTYPE                          object
BSL_SB_CUST                       object
BSL_CUST_AIRB                     object
HasAIRBProduct                    object
Widely_Held                       object
NFP                               object
Out_of_Country                    object
SPP_Group_1                       object
SPP_Group_2                       object
Automotive_New_Cars_Dealer        object
BOBB                              object
OPFBCUST                          object
DOCTOR_DENTIST_IND                object
RESTAURANT_IND                    object
TRUCKING_IND    

- tu_seq_id: Customer ID
- TIME_KEY(numeric categorical): 15076 = Jan 2014, 15196 = Apr 2014, 15316 = July 2014, 15436 = Oct 2014
- 'CUSTTYPE' - 'SPP_LISTED_OR_TARGETED_IND'：all text categorical variables
- All variables after are numerical (some are categorical variables)
- 't1' to 't12' should be used to create the response variable
- 'GRP_ALL2320' - 'WOE_max_ks_num_dlqdays_12mos' contain WOE calculations for many variables....
- credit and debit variables are used to create new explanatory variables
- 4 benchmark variables at the end that might need to be separated out


In [158]:
# Create new columns indicating the months difference between the latest depost/account open date and the 'TIME_KEY' date
# -- function to standardize the 'TIME_KEY' column
def time_key(column):
    map_time_key  = {15076:201401, 15196:201404, 15316:201407, 15436:201410}
    return column.map(map_time_key)

# -- function to convert integer date column to datetime type and calculate the difference from the date to the 
# 'TIME_KEY' date --
def date_diff(df, column):
    df[column] = pd.to_datetime(df[column], format= '%Y%m')
    # calculate the difference in months and add to a new column, drop the original column
    df['mths_from_' + column] = round(pd.to_numeric((df['TIME_KEY'] - df[column]) / np.timedelta64(1, 'M')))
    df.drop(columns = [column], axis=1)

# standardize the 'Time_Key' column
df_time_key = time_key(df['TIME_KEY'])
df_time_key = pd.to_datetime(df_time_key, format= '%Y%m')
df['TIME_KEY'] = df_time_key

# deal with other date columns
df['Go129_yymm'] = date_diff(df, 'Go129_yymm')
df['dda_max_open_date'] = date_diff(df, 'dda_max_open_date')
df['dda_max_Last_Dep_Date'] = date_diff(df, 'dda_max_Last_Dep_Date')
# drop the 'mths_from_Go129_yymm' column b/c it only has the value 6. No effect on the modeling later
df.drop(columns = ['mths_from_Go129_yymm'], inplace = True)

In [159]:
# create new variables based on the credit and debit variables
df['liability_curr'] = df['credit_curr'] - df['debit_curr']
for i in range(12):
    df['liability_prev' + str(i+1)] = df['credit_prev' + str(i+1)] - df['debit_prev' + str(i+1)]

# drop the original credit and debit variables
index = df.columns.get_loc('debit_curr')
df.drop(df.columns[index:index+26], axis=1, inplace=True)
df['liability_curr']

0      -22509.962140
1      -34142.288207
2       -6102.942330
3      -15447.382545
4         164.766700
5       24309.611310
6       -3818.855600
7        2453.038819
8      -30624.435090
9        9000.886800
10     -16102.250791
11     -12588.006260
12      30547.868126
13      12782.152470
14      31474.624191
15      -9829.182530
16      -4407.212390
17       3731.016950
18       -262.912590
19      12554.103940
20      -3183.859820
21     -19346.546160
22      23202.719370
23      29055.745420
24     -19242.554430
25     -14157.820333
26      -3742.425450
27     -18469.828700
28      25290.366390
29      -8653.592810
30      -2092.424970
31     -28497.825921
32       7869.006810
33      22109.381980
34      14796.067660
35      -8199.359040
36     -17963.029410
37      -1401.486780
38      -4610.128120
39       8111.173780
40     -17788.407218
41      -4049.346000
42     -15976.338350
43     -16880.219390
44     -17847.378670
45      16644.542620
46     -25640.569616
47      -5941

In [160]:
# -- function to create bins for the liability variables --
def bin_liability(df, column):
    df[column] = np.where((df[column]>0), 1, 0)

# create bin for liability variables
bin_liability(df, 'liability_curr')
for i in range(12):
    bin_liability(df, 'liability_prev' + str(i+1))

In [161]:
# delete widely held customers and deceased customers
n_pop = df.shape[0]
index = (df['WIDELYHD'] == 'Y') | (df['deceased'] == 1)
df = df.drop(df.index[index])
print(n_pop)
print(len(df.index))
print(n_pop-len(df.index))

9028
9012
16


In [162]:
# extract the 4 benchmark variables
df_benchmark = df[['benchmark1', 'benchmark2', 'benchmark3', 'benchmark4']]
df.drop(columns = ['benchmark1', 'benchmark2', 'benchmark3', 'benchmark4'], axis=1, inplace=True)
# extract the 'TIME_KEY' variable
df_timekey = df['TIME_KEY']
df.drop(columns = ['TIME_KEY'], inplace = True)


- Create the response variable \
Observed that if t_i is 1, then t_i, t_i+1,..., t_12 are all 1


In [163]:
# Create response variable
# bin=0: no default behavior; bin=12: defaulted for 12 months
index = df.columns.get_loc('t1')
df['default_risk'] = df.iloc[:, index:index+12].sum(axis=1)

# Calculate the default rate for the entire population as well as for each observation point
a = {}
b = {}
for i in range(12):
    a["pct_default_t{0}".format(i+1)] = (df['t' + str(i+1)] != 0).sum() / df.shape[0]
    b["count_default_t{0}".format(i+1)] = (df['t' + str(i+1)] != 0).sum()
a,b

({'pct_default_t1': 0.009764758100310697,
  'pct_default_t2': 0.019529516200621395,
  'pct_default_t3': 0.02785175321793165,
  'pct_default_t4': 0.03595206391478029,
  'pct_default_t5': 0.04360852197070573,
  'pct_default_t6': 0.050821127385707945,
  'pct_default_t7': 0.05770084332001776,
  'pct_default_t8': 0.06657789613848203,
  'pct_default_t9': 0.07334664891256103,
  'pct_default_t10': 0.08288948069241012,
  'pct_default_t11': 0.091988459831336,
  'pct_default_t12': 0.09986684420772303},
 {'count_default_t1': 88,
  'count_default_t2': 176,
  'count_default_t3': 251,
  'count_default_t4': 324,
  'count_default_t5': 393,
  'count_default_t6': 458,
  'count_default_t7': 520,
  'count_default_t8': 600,
  'count_default_t9': 661,
  'count_default_t10': 747,
  'count_default_t11': 829,
  'count_default_t12': 900})

In [164]:
# drop the original t1 - t12 variables
index = df.columns.get_loc('t1')
df.drop(df.columns[index:index+12], axis=1, inplace=True)
df['default_risk'].head(10)

0    0
1    0
2    0
3    0
4    0
5    0
6    5
7    3
8    0
9    0
Name: default_risk, dtype: int64

In [165]:
# Check for missing data
NAN = [(c,df[c].isnull().mean().round(4)*100) for c in df]
NAN = pd.DataFrame(NAN, columns=['column_name','percentage missing(%)'])
print(NAN)  # No column has a significant amount of missing values

                         column_name  percentage missing(%)
0                          tu_seq_id                   0.00
1                           CUSTTYPE                   0.00
2                           HIGHRISK                   0.00
3                           WIDELYHD                   0.00
4                            STARTUP                   0.00
5                           TERMCUST                   0.00
6                            PDLCUST                   0.00
7                             OLCUST                   0.00
8                           VISACUST                   0.00
9                           PROFTYPE                   0.00
10                       BSL_SB_CUST                   0.00
11                     BSL_CUST_AIRB                   0.00
12                    HasAIRBProduct                   0.00
13                       Widely_Held                   0.01
14                               NFP                   0.00
15                    Out_of_Country    

In [166]:
# drop columns that have more than 80% of the data missing
index = NAN.iloc[:,1] >= 80
df = df.drop(NAN.iloc[:,0][index], axis=1)
# drop the customer id variable
df = df.drop(['tu_seq_id'], axis=1)
len(df.columns) 

405

In [167]:
# one-hot encoding for text categorical variables
df_text = df.select_dtypes(include=['object'])
df_text_dummies = pd.get_dummies(df_text)
df_text_dummies.head()

,CUSTTYPE_B,CUSTTYPE_O,CUSTTYPE_T,HIGHRISK_N,HIGHRISK_Y,WIDELYHD_N,STARTUP_N,STARTUP_Y,TERMCUST_N,TERMCUST_Y,PDLCUST_N,OLCUST_N,OLCUST_Y,VISACUST_N,VISACUST_Y,PROFTYPE_C,BSL_SB_CUST_Y,BSL_CUST_AIRB_Y,HasAIRBProduct_Y,Widely_Held_N,NFP_N,Out_of_Country_N,Out_of_Country_Y,SPP_Group_1_N,SPP_Group_1_Y,SPP_Group_2_N,SPP_Group_2_Y,Automotive_New_Cars_Dealer_N,OPFBCUST_N,DOCTOR_DENTIST_IND_N,DOCTOR_DENTIST_IND_Y,RESTAURANT_IND_N,RESTAURANT_IND_Y,TRUCKING_IND_N,TRUCKING_IND_Y,RESTAURANT_OR_TRUCKING_IND_N,RESTAURANT_OR_TRUCKING_IND_Y,SPP_LISTED_IND_N,SPP_LISTED_IND_Y,SPP_TARGETED_IND_N,SPP_TARGETED_IND_Y,SPP_LISTED_OR_TARGETED_IND_N,SPP_LISTED_OR_TARGETED_IND_Y
0,0,1,0,1,0,1,1,0,1,0,1,1,0,0,1,1,1,1,1,1,1,1,0,1,0,1,0,1,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0
1,0,1,0,1,0,1,1,0,1,0,1,1,0,1,0,1,1,1,1,1,1,1,0,0,1,1,0,1,1,0,1,1,0,1,0,1,0,1,0,0,1,0,1
2,0,1,0,1,0,1,1,0,1,0,1,1,0,0,1,1,1,1,1,1,1,1,0,1,0,1,0,1,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0
3,0,1,0,1,0,1,1,0,1,0,1,1,0,1,0,1,1,1,1,1,1,1,0,1,0,1,0,1,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0
4,0,1,0,1,0,1,1,0,1,0,1,1,0,0,1,1,1,1,1,1,1,1,0,1,0,1,0,1,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0


In [168]:
# drop the old text categorical variables and append the newly created numeric ones
df.drop(df.columns[0:26], axis = 1, inplace = True)
df = pd.concat([df, df_text_dummies], axis=1)
df.head()

,TBSAT01,TBSAT103S,TBSAT33A,TBSAT34B,TBSBC104S,TBSBC33S,TBSBC35S,TBSBC97A,TBSBR34S,TBSG001B,TBSG059S,TBSG202A,TBSG302S,TBSRE24S,TBSRE29S,TBSRE30S,TBSRE36S,TBSRN33S,TBSRN34S,GO170,NA11,CVPRAEP112,CVPRAGG102,CVPRAGG501,CVPRAGG512,CVPRAGG519,CVPRAGG905,CVPRAGG907,CVPRAGG910,CVPRRVLR01,CVPRRVLR07,CVPRTPR103,CVPRTPR212,CVPRTPR301,CVPRTPR312,CVPRTRV04,CVPRTRV12,CVPRTRV14,CVPRWALSHR01,CVPRWALSHR02,cust_any_nsf,cust_card_max_util_6mos,cust_max_dlq_3mos,cust_rev_max_dlq_6mos,cust_sum_dlq_24mos,cust_sum_dlqdays_12mos,d2dbalp_avg_l6m,agri_cust,TBSSC100,CVPRRVLR29,GO138,dda_sum_ABM_Dep_Txn_Cnt,dda_sum_Acc_Cr_Bal,dda_sum_Acc_Daily_OD_Int,dda_sum_Acc_Db_Bal,dda_max_Avg_Cr_Bal,dda_max_Avg_Cr_Bal_Prev,dda_min_Avg_Db_Bal,dda_sum_Chargebacks,dda_sum_Dep_Txn_Cnt,dda_sum_Limit,dda_max_Max_Mthly_Bal,dda_min_Min_Mthly_Bal,dda_sum_Night_Dep_Txn_Cnt,dda_sum_OD_Charges,dda_sum_OL_Days,dda_sum_OS_Bal,dda_sum_Qtr_NSF,dda_sum_Qtr_NSF_Prev,dda_sum_Qtr_NSF_Prev_2,dda_sum_Qtr_NSF_Prev_3,dda_sum_Ttl_Dep_Prev,dda_sum_Ttl_Dep_Prev_2,dda_sum_Ttl_Dep_Prev_3,dda_avg_av_db_bal,dda_time_since_open_mos,dda_max_mthly_av_bal,dda_min_mthly_av_bal,dda_av_bal,dda_time_since_dep_days,dda_avg_dly_dep_amt_L90,dda_ratio_cr_bal,ks_tot_limit,ks_tot_osbal,max_ks_any_def_24mos,max_ks_any_def_12mos,max_ks_any_def_6mos,max_ks_any_def_3mos,max_ks_any_bad_24mos,max_ks_any_bad_12mos,max_ks_any_bad_6mos,max_ks_any_bad_3mos,max_ks_max_dlqdays_24mos,max_ks_max_dlqdays_12mos,max_ks_max_dlqdays_6mos,max_ks_max_dlqdays_3mos,max_ks_num_def_24mos,max_ks_num_def_12mos,max_ks_num_def_6mos,max_ks_num_def_3mos,max_ks_num_bad_24mos,max_ks_num_bad_12mos,max_ks_num_bad_6mos,max_ks_num_bad_3mos,max_ks_num_dlq_24mos,max_ks_num_dlq_12mos,max_ks_num_dlq_6mos,max_ks_num_dlq_3mos,max_ks_num_dlqdays_24mos,max_ks_num_dlqdays_12mos,max_ks_num_dlqdays_6mos,max_ks_num_dlqdays_3mos,max_ks_num_overlimit_24mos,max_ks_num_overlimit_12mos,max_ks_num_overlimit_6mos,max_ks_num_overlimit_3mos,max_ks_max_dlq_24mos,max_ks_max_dlq_12mos,max_ks_max_dlq_6mos,max_ks_max_dlq_3mos,max_ks_age,max_ks_max_util_24mos,max_ks_max_util_12mos,max_ks_max_util_6mos,max_ks_max_util_3mos,pct_ks_util,ALL2320,ALL2326,ALL2327,ALL2350,ALL2358,ALL2380,ALL2700,ALL6160,ALL6200,ALL6230,ALL7330,ALL7938,ALL8150,ALL8160,ALL8358,BCA2358,BCA2380,BCA5030,BCC3510,BCC3515,BCC5620,BCC5830,BCC6200,BCC6280,BCC7110,BCC7120,BRC8158,HLC5030,HLC7110,IQT9410,IQT9420,REV2320,REV2327,REV2328,REV2350,REV3423,REV5020,REV5030,REV5620,REV8153,GRP_ALL2320,WOE_ALL2320,GRP_ALL2326,WOE_ALL2326,GRP_ALL2327,WOE_ALL2327,GRP_ALL2350,WOE_ALL2350,GRP_ALL2358,WOE_ALL2358,GRP_ALL2380,WOE_ALL2380,GRP_ALL2700,WOE_ALL2700,GRP_ALL6160,WOE_ALL6160,GRP_ALL6200,WOE_ALL6200,GRP_ALL6230,WOE_ALL6230,GRP_ALL7330,WOE_ALL7330,GRP_ALL7938,WOE_ALL7938,GRP_ALL8150,WOE_ALL8150,GRP_ALL8160,WOE_ALL8160,GRP_ALL8358,WOE_ALL8358,GRP_BCA2358,WOE_BCA2358,GRP_BCA2380,WOE_BCA2380,GRP_BCA5030,WOE_BCA5030,GRP_BCC3510,WOE_BCC3510,GRP_BCC3515,WOE_BCC3515,GRP_BCC5620,WOE_BCC5620,GRP_BCC5830,WOE_BCC5830,GRP_BCC6200,WOE_BCC6200,GRP_BCC6280,WOE_BCC6280,GRP_BCC7110,WOE_BCC7110,GRP_BCC7120,WOE_BCC7120,GRP_BRC8158,WOE_BRC8158,GRP_CVPRAEP112,WOE_CVPRAEP112,GRP_CVPRAGG102,WOE_CVPRAGG102,GRP_CVPRAGG501,WOE_CVPRAGG501,GRP_CVPRAGG512,WOE_CVPRAGG512,GRP_CVPRAGG519,WOE_CVPRAGG519,GRP_CVPRAGG905,WOE_CVPRAGG905,GRP_CVPRAGG907,WOE_CVPRAGG907,GRP_CVPRAGG910,WOE_CVPRAGG910,GRP_CVPRRVLR01,WOE_CVPRRVLR01,GRP_CVPRRVLR07,WOE_CVPRRVLR07,GRP_CVPRTPR103,WOE_CVPRTPR103,GRP_CVPRTPR212,WOE_CVPRTPR212,GRP_CVPRTPR301,WOE_CVPRTPR301,GRP_CVPRTPR312,WOE_CVPRTPR312,GRP_CVPRTRV04,WOE_CVPRTRV04,GRP_CVPRTRV12,WOE_CVPRTRV12,GRP_CVPRTRV14,WOE_CVPRTRV14,GRP_CVPRWALSHR01,WOE_CVPRWALSHR01,GRP_CVPRWALSHR02,WOE_CVPRWALSHR02,GRP_CVSC100,WOE_CVSC100,GRP_CVSC110,WOE_CVSC110,GRP_GO170,WOE_GO170,GRP_HLC5030,WOE_HLC5030,GRP_HLC7110,WOE_HLC7110,GRP_IQT9410,WOE_IQT9410,GRP_IQT9420,WOE_IQT9420,GRP_NA11,WOE_NA11,GRP_PD_Total_Scorecard_Points,WOE_PD_Total_Scorecard_Points,GRP_REV2320,WOE_REV2320,GRP_REV2327,WOE_REV2327,GRP_REV2328,WOE_REV2328,GRP_REV2350,WOE_REV

Now all variables are numeric except for missing values

In [169]:
# Normalizing numeric variables
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
df = pd.DataFrame(sc.fit_transform(df), columns=df.columns)
df.head(10)

,TBSAT01,TBSAT103S,TBSAT33A,TBSAT34B,TBSBC104S,TBSBC33S,TBSBC35S,TBSBC97A,TBSBR34S,TBSG001B,TBSG059S,TBSG202A,TBSG302S,TBSRE24S,TBSRE29S,TBSRE30S,TBSRE36S,TBSRN33S,TBSRN34S,GO170,NA11,CVPRAEP112,CVPRAGG102,CVPRAGG501,CVPRAGG512,CVPRAGG519,CVPRAGG905,CVPRAGG907,CVPRAGG910,CVPRRVLR01,CVPRRVLR07,CVPRTPR103,CVPRTPR212,CVPRTPR301,CVPRTPR312,CVPRTRV04,CVPRTRV12,CVPRTRV14,CVPRWALSHR01,CVPRWALSHR02,cust_any_nsf,cust_card_max_util_6mos,cust_max_dlq_3mos,cust_rev_max_dlq_6mos,cust_sum_dlq_24mos,cust_sum_dlqdays_12mos,d2dbalp_avg_l6m,agri_cust,TBSSC100,CVPRRVLR29,GO138,dda_sum_ABM_Dep_Txn_Cnt,dda_sum_Acc_Cr_Bal,dda_sum_Acc_Daily_OD_Int,dda_sum_Acc_Db_Bal,dda_max_Avg_Cr_Bal,dda_max_Avg_Cr_Bal_Prev,dda_min_Avg_Db_Bal,dda_sum_Chargebacks,dda_sum_Dep_Txn_Cnt,dda_sum_Limit,dda_max_Max_Mthly_Bal,dda_min_Min_Mthly_Bal,dda_sum_Night_Dep_Txn_Cnt,dda_sum_OD_Charges,dda_sum_OL_Days,dda_sum_OS_Bal,dda_sum_Qtr_NSF,dda_sum_Qtr_NSF_Prev,dda_sum_Qtr_NSF_Prev_2,dda_sum_Qtr_NSF_Prev_3,dda_sum_Ttl_Dep_Prev,dda_sum_Ttl_Dep_Prev_2,dda_sum_Ttl_Dep_Prev_3,dda_avg_av_db_bal,dda_time_since_open_mos,dda_max_mthly_av_bal,dda_min_mthly_av_bal,dda_av_bal,dda_time_since_dep_days,dda_avg_dly_dep_amt_L90,dda_ratio_cr_bal,ks_tot_limit,ks_tot_osbal,max_ks_any_def_24mos,max_ks_any_def_12mos,max_ks_any_def_6mos,max_ks_any_def_3mos,max_ks_any_bad_24mos,max_ks_any_bad_12mos,max_ks_any_bad_6mos,max_ks_any_bad_3mos,max_ks_max_dlqdays_24mos,max_ks_max_dlqdays_12mos,max_ks_max_dlqdays_6mos,max_ks_max_dlqdays_3mos,max_ks_num_def_24mos,max_ks_num_def_12mos,max_ks_num_def_6mos,max_ks_num_def_3mos,max_ks_num_bad_24mos,max_ks_num_bad_12mos,max_ks_num_bad_6mos,max_ks_num_bad_3mos,max_ks_num_dlq_24mos,max_ks_num_dlq_12mos,max_ks_num_dlq_6mos,max_ks_num_dlq_3mos,max_ks_num_dlqdays_24mos,max_ks_num_dlqdays_12mos,max_ks_num_dlqdays_6mos,max_ks_num_dlqdays_3mos,max_ks_num_overlimit_24mos,max_ks_num_overlimit_12mos,max_ks_num_overlimit_6mos,max_ks_num_overlimit_3mos,max_ks_max_dlq_24mos,max_ks_max_dlq_12mos,max_ks_max_dlq_6mos,max_ks_max_dlq_3mos,max_ks_age,max_ks_max_util_24mos,max_ks_max_util_12mos,max_ks_max_util_6mos,max_ks_max_util_3mos,pct_ks_util,ALL2320,ALL2326,ALL2327,ALL2350,ALL2358,ALL2380,ALL2700,ALL6160,ALL6200,ALL6230,ALL7330,ALL7938,ALL8150,ALL8160,ALL8358,BCA2358,BCA2380,BCA5030,BCC3510,BCC3515,BCC5620,BCC5830,BCC6200,BCC6280,BCC7110,BCC7120,BRC8158,HLC5030,HLC7110,IQT9410,IQT9420,REV2320,REV2327,REV2328,REV2350,REV3423,REV5020,REV5030,REV5620,REV8153,GRP_ALL2320,WOE_ALL2320,GRP_ALL2326,WOE_ALL2326,GRP_ALL2327,WOE_ALL2327,GRP_ALL2350,WOE_ALL2350,GRP_ALL2358,WOE_ALL2358,GRP_ALL2380,WOE_ALL2380,GRP_ALL2700,WOE_ALL2700,GRP_ALL6160,WOE_ALL6160,GRP_ALL6200,WOE_ALL6200,GRP_ALL6230,WOE_ALL6230,GRP_ALL7330,WOE_ALL7330,GRP_ALL7938,WOE_ALL7938,GRP_ALL8150,WOE_ALL8150,GRP_ALL8160,WOE_ALL8160,GRP_ALL8358,WOE_ALL8358,GRP_BCA2358,WOE_BCA2358,GRP_BCA2380,WOE_BCA2380,GRP_BCA5030,WOE_BCA5030,GRP_BCC3510,WOE_BCC3510,GRP_BCC3515,WOE_BCC3515,GRP_BCC5620,WOE_BCC5620,GRP_BCC5830,WOE_BCC5830,GRP_BCC6200,WOE_BCC6200,GRP_BCC6280,WOE_BCC6280,GRP_BCC7110,WOE_BCC7110,GRP_BCC7120,WOE_BCC7120,GRP_BRC8158,WOE_BRC8158,GRP_CVPRAEP112,WOE_CVPRAEP112,GRP_CVPRAGG102,WOE_CVPRAGG102,GRP_CVPRAGG501,WOE_CVPRAGG501,GRP_CVPRAGG512,WOE_CVPRAGG512,GRP_CVPRAGG519,WOE_CVPRAGG519,GRP_CVPRAGG905,WOE_CVPRAGG905,GRP_CVPRAGG907,WOE_CVPRAGG907,GRP_CVPRAGG910,WOE_CVPRAGG910,GRP_CVPRRVLR01,WOE_CVPRRVLR01,GRP_CVPRRVLR07,WOE_CVPRRVLR07,GRP_CVPRTPR103,WOE_CVPRTPR103,GRP_CVPRTPR212,WOE_CVPRTPR212,GRP_CVPRTPR301,WOE_CVPRTPR301,GRP_CVPRTPR312,WOE_CVPRTPR312,GRP_CVPRTRV04,WOE_CVPRTRV04,GRP_CVPRTRV12,WOE_CVPRTRV12,GRP_CVPRTRV14,WOE_CVPRTRV14,GRP_CVPRWALSHR01,WOE_CVPRWALSHR01,GRP_CVPRWALSHR02,WOE_CVPRWALSHR02,GRP_CVSC100,WOE_CVSC100,GRP_CVSC110,WOE_CVSC110,GRP_GO170,WOE_GO170,GRP_HLC5030,WOE_HLC5030,GRP_HLC7110,WOE_HLC7110,GRP_IQT9410,WOE_IQT9410,GRP_IQT9420,WOE_IQT9420,GRP_NA11,WOE_NA11,GRP_PD_Total_Scorecard_Points,WOE_PD_Total_Scorecard_Points,GRP_REV2320,WOE_REV2320,GRP_REV2327,WOE_REV2327,GRP_REV2328,WOE_REV2328,GRP_REV2350,WOE_REV

In [170]:
# Deal with missing values for all variables using the KNN method
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
df = pd.DataFrame(imputer.fit_transform(df),columns = df.columns)
df_dummy = pd.DataFrame(sc.inverse_transform(df), columns=df.columns)
df['default_risk'] = df_dummy['default_risk']

In [171]:
df.isna().any()

TBSAT01                            False
TBSAT103S                          False
TBSAT33A                           False
TBSAT34B                           False
TBSBC104S                          False
TBSBC33S                           False
TBSBC35S                           False
TBSBC97A                           False
TBSBR34S                           False
TBSG001B                           False
TBSG059S                           False
TBSG202A                           False
TBSG302S                           False
TBSRE24S                           False
TBSRE29S                           False
TBSRE30S                           False
TBSRE36S                           False
TBSRN33S                           False
TBSRN34S                           False
GO170                              False
NA11                               False
CVPRAEP112                         False
CVPRAGG102                         False
CVPRAGG501                         False
CVPRAGG512      

No missing values

In [172]:
# Check the variance for all features
print(df.var().sort_values(kind="quicksort",ascending=True))
# drop the ones that have 0 variance (constant values)
df.drop(['NFP_N', 'deceased', 'dda_sum_OL_Days', 'BSL_SB_CUST_Y', 'PROFTYPE_C', 'PDLCUST_N', 'HasAIRBProduct_Y', \
              'WIDELYHD_N', 'BSL_CUST_AIRB_Y', 'OPFBCUST_N', 'Automotive_New_Cars_Dealer_N'], axis=1, inplace=True)

NFP_N                              0.000000
deceased                           0.000000
dda_sum_OL_Days                    0.000000
BSL_SB_CUST_Y                      0.000000
PROFTYPE_C                         0.000000
PDLCUST_N                          0.000000
HasAIRBProduct_Y                   0.000000
WIDELYHD_N                         0.000000
BSL_CUST_AIRB_Y                    0.000000
OPFBCUST_N                         0.000000
Automotive_New_Cars_Dealer_N       0.000000
cust_any_nsf                       0.712930
dda_ratio_cr_bal                   0.719899
max_ks_num_def_3mos                0.741152
max_ks_any_def_3mos                0.741152
max_ks_num_def_6mos                0.741238
max_ks_any_def_6mos                0.741238
max_ks_num_def_12mos               0.760907
max_ks_any_def_12mos               0.760907
max_ks_num_bad_6mos                0.765074
max_ks_any_def_24mos               0.766517
max_ks_num_bad_12mos               0.767676
max_ks_num_overlimit_24mos      

In [173]:
print(df.nunique())

TBSAT01                              22
TBSAT103S                            19
TBSAT33A                           4903
TBSAT34B                            118
TBSBC104S                           120
TBSBC33S                           2885
TBSBC35S                           2825
TBSBC97A                           2887
TBSBR34S                            122
TBSG001B                             20
TBSG059S                              7
TBSG202A                           4503
TBSG302S                             12
TBSRE24S                             16
TBSRE29S                             15
TBSRE30S                             23
TBSRE36S                             29
TBSRN33S                           2211
TBSRN34S                            122
GO170                               505
NA11                                422
CVPRAEP112                         4882
CVPRAGG102                         8152
CVPRAGG501                         6183
CVPRAGG512                         6029


- Assume features with less or equal to 15 unique values are categorical. Others are all numeric features

In [174]:
# select the best categorical and numeric features
from scipy.stats import chi2_contingency
from sklearn.feature_selection import f_classif

# separate categorical and numeric features
index_categ = df.nunique()<=15
index_numeric = df.nunique()>15
y = df['default_risk']
df_categ = df[df.columns[index_categ]]
df_categ = df_categ.drop(['default_risk'], axis=1)
df_numeric = df[df.columns[index_numeric]]

# categorical:
chi_result = {}  # store the chi-square test result

for column in df_categ:
    chi, p, dof, ex = chi2_contingency(pd.crosstab(y, df_categ[column]))
    chi_result.setdefault('Categorical Features',[]).append(column)
    chi_result.setdefault('p-value',[]).append(round(p, 10))

# convert the dictionary to a DF
chi_result = pd.DataFrame(data = chi_result)
chi_result.sort_values(by = ['p-value'], ascending = True, ignore_index = True, inplace = True)
print(chi_result)
# Select all categorical variables with 0 p-value
index = chi_result['p-value']==0
df_categ_sel = df_categ[chi_result['Categorical Features'][index]]

              Categorical Features       p-value
0                         TBSG059S  0.000000e+00
1                      WOE_REV2327  0.000000e+00
2                      GRP_REV2328  0.000000e+00
3                      WOE_REV2328  0.000000e+00
4                      GRP_REV2350  0.000000e+00
5                      WOE_REV2350  0.000000e+00
6                      GRP_REV3423  0.000000e+00
7                      WOE_REV3423  0.000000e+00
8                      GRP_REV5020  0.000000e+00
9                      WOE_REV5020  0.000000e+00
10                     GRP_REV5620  0.000000e+00
11                     WOE_REV5620  0.000000e+00
12                     GRP_REV2327  0.000000e+00
13                     GRP_REV8153  0.000000e+00
14                   GRP_TBSAT103S  0.000000e+00
15                   WOE_TBSAT103S  0.000000e+00
16                    GRP_TBSAT33A  0.000000e+00
17                    WOE_TBSAT33A  0.000000e+00
18                    GRP_TBSAT34B  0.000000e+00
19                  

In [175]:
# numeric:
# Calculate F Statistic and the corresponding p values
F_stat, p_value = f_classif(df_numeric, y)

# convert to a DF
ANOVA = pd.DataFrame(data = {'Numerical Features': df_numeric.columns.values, \
                'F-value': F_stat, 'p-value': p_value.round(decimals=10)})
ANOVA.sort_values(by = ['F-value'], ascending = False, ignore_index = True, inplace = True)
print(ANOVA)

# select the features with F-value > 6
index = ANOVA['F-value']>6
df_num_sel=df_numeric[ANOVA['Numerical Features'][index]]

                  Numerical Features     F-value       p-value
0              cust_rev_max_dlq_6mos  147.700570  0.000000e+00
1                 cust_sum_dlq_24mos  104.364685  0.000000e+00
2            max_ks_num_dlqdays_3mos   90.691551  0.000000e+00
3            max_ks_max_dlqdays_6mos   88.227233  0.000000e+00
4            max_ks_max_dlqdays_3mos   87.416844  0.000000e+00
5            max_ks_num_dlqdays_6mos   84.447420  0.000000e+00
6           max_ks_max_dlqdays_12mos   84.111259  0.000000e+00
7           max_ks_num_dlqdays_12mos   81.429136  0.000000e+00
8             cust_sum_dlqdays_12mos   80.905012  0.000000e+00
9           max_ks_max_dlqdays_24mos   68.336746  0.000000e+00
10              max_ks_num_dlq_12mos   67.632770  0.000000e+00
11               max_ks_num_dlq_3mos   64.200554  0.000000e+00
12              max_ks_num_dlq_24mos   60.499944  0.000000e+00
13          max_ks_num_dlqdays_24mos   56.660462  0.000000e+00
14         max_ks_num_overlimit_6mos   55.374565  0.000

In [176]:
print(len(df_categ_sel.columns))
print(len(df_num_sel.columns))

219
65


In [187]:
# Remove features that might cause multicollinearity issues
df = pd.concat([df_categ_sel, df_num_sel, y], axis=1,sort=False)
cor = df.corr().abs()
# Select upper/lower triangle of correlation matrix
upper_triangle = cor.where(np.triu(np.ones(cor.shape), k=1).astype(np.bool))

# Find features with correlation greater than 0.85
drop_lst1 = [column for column in upper_triangle.columns if any(upper_triangle[column] > 0.85)]

# Find features with correlation to the reponse smaller than 0.1
drop_lst2 = [column for column in cor.columns if cor[column][-1] < 0.1]

print(len(drop_lst1))
# Drop features 
df.drop(drop_lst1+drop_lst2, axis=1, inplace=True)
print(len(df.columns))

148
107


- Train/validation split (two methodology)

In [188]:
# train_test_split, 80% training
X = df.drop(['default_risk'], axis=1)
y = df['default_risk']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, \
                                                    random_state = 42, stratify = y)

In [189]:
# Out-of-time test sample: test sample is as of 2014/10/01, while training are as of 2014/1/1, 2014/4/1 or 2014/7/1
import datetime
df_with_time = pd.concat([df_timekey,df], axis=1,sort=False)
dates = pd.to_datetime([datetime.datetime(2014, 1, 1), datetime.datetime(2014, 4, 1), datetime.datetime(2014, 7, 1)])

df_train = df_with_time.loc[df_with_time['TIME_KEY'].isin(dates)]
df_test = df_with_time.loc[df_with_time['TIME_KEY'] == datetime.datetime(2014, 10, 1)]

X_train1 = df_train.drop(['default_risk'], axis=1)
X_test1 = df_test.drop(['default_risk'], axis=1)
y_train1 = df_train['default_risk']
y_test1 = df_test['default_risk']
